# Compound interest

This is a Jupyter Voilà example showing how a minimalist webapplication looks like. You can hardly change the design, but content can be easily modified or added if you have access to the underyling Jupyter notebook. Voilà turns a Jupyter notebook into a webapplication with the help of Jupyter widgets which can potentially be used in onsite Jupyter notebooks too. Jupyter notebooks are usually not exposed to Internet-public pages, because arbitrary codes could be executed, which is dangerous for the server owner. Jupyter widgets give the possibility to modify vairables without exposing the source code, limiting the interaction between the user and the server to some selected operation, thus solving the security issue of exposed Jupyter notebooks.

In my [other flask example](http://tuzes.elte.hu:5000/calc) I have full control over the design, content and code, but it also means that I have to provide the glue between

- HTML request,
- python calculation and
- rendering the result
- return server response

Here I only have to have a python environment with the proper python packages, wite the python code, and nothing else (wqeb query, webserver, HTML) needs to be taken care of.

In [59]:
from ipywidgets import FloatText, IntText, Layout, HBox, VBox, Label, Button, Output, HTML
from IPython.display import display, Markdown, clear_output
import numpy
import matplotlib.pyplot as plt

params = {
    "interest_rate": [1., "Interest rate (%)"],
    "yearly_savings": [10_000., "Yearly savings ($)"],
    "term": [10, "Term (year)"],
    "initial_saving": [100_000., "Initial saving ($)"]
    }

output = Output()

mwidgets = {}
for name, param in params.items():
    if isinstance(param[0], float):
        mwidgets[name] = FloatText(
            value=param[0],
            disabled=False,
            )
    elif isinstance(param[0], int):
        mwidgets[name] = IntText(
            value=param[0],
            disabled=False
            )
    
    display(HBox([Label(param[1], layout=Layout(width="10em", display="flex", justify_content="flex-end")),mwidgets[name]]))

def recalculate(arg):
    with output:
        clear_output()
        
        savings = [mwidgets["initial_saving"].value,
                   mwidgets["initial_saving"].value * (1 + mwidgets["interest_rate"].value/100)]
        term = int(mwidgets["term"].value)

        for _ in range(0, term-1):
            savings.append(
                savings[-1] * (1 + mwidgets["interest_rate"].value/100) + mwidgets["yearly_savings"].value)
        
        display(Markdown("Total savings ($): {:.0f}".format(savings[-1])))
        table = """
## Explanation:
        
| time (year) | Actual saving ($)|
| ----------: |:-----------------|
"""

        table += "| Initial | {:.0f}|\n".format(savings[0])
        for year, saving in enumerate(savings[1:]):
            table += "| {} | {:.0f}|\n".format(year+1,saving)
        display(Markdown(table))
        years = [*range(0, mwidgets["term"].value+1)]
        labels = years[:]
        labels[0] = "initial"
        plt.clf()

        plt.suptitle('Total savings at the end of the years')
        plt.xticks(ticks=years, labels=labels)

        plt.ylabel('Actual saving ($1000)')
        plt.xlabel('time (year)')
        
        savings = numpy.array(savings)
        plt.bar(years, savings/1000)

        plt.show()
        
button = Button(description="Recalculate!")
button.on_click(recalculate)
VBox([button,output])